## Chuẩn bị dữ liệu SER với bộ ViSEC

Notebook này thực hiện các bước tiền xử lý cho bài toán Nhận diện cảm xúc trong giọng nói (SER) sử dụng bộ dữ liệu `hustep-lab/ViSEC`:

1. Tải bộ dữ liệu ViSEC từ Hugging Face.
2. Chuẩn hóa nhãn cảm xúc từ 4 lớp gốc (`happy`, `neutral`, `sad`, `angry`) về 3 lớp:
   - `positive`: happy
   - `neutral`: neutral
   - `negative`: sad, angry
3. Phân tích phân bố nhãn cảm xúc, accent, giới tính.
4. Chia dữ liệu thành 3 tập:
   - `train`: 70%
   - `validation`: 15%
   - `test`: 15%
   với ràng buộc stratify theo nhãn cảm xúc.
5. Chuẩn hóa cột `audio` về dạng `Audio(sampling_rate=16000)`.
6. Lưu lại các tập dữ liệu dưới dạng Hugging Face Dataset bằng `save_to_disk` tại thư mục `./data/ser_visec`.
7. Ghi thống kê cơ bản ra file `stats.json` để phục vụ báo cáo và huấn luyện sau này.


In [ ]:
# Cài đặt thư viện cần thiết (chạy một lần trong môi trường mới)
# Nếu đã có rồi thì có thể bỏ qua cell này
%pip install -q datasets soundfile librosa tqdm

from datasets import load_dataset, Dataset, Audio
from pathlib import Path
from sklearn.model_selection import train_test_split
import pandas as pd
import json

# Thư mục lưu dataset SER đã xử lý
OUTPUT_DIR = Path("./data/ser_visec")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Thư mục lưu dữ liệu:", OUTPUT_DIR.resolve())

# 1. Tải dataset ViSEC từ Hugging Face
print("\n[1/5] Đang tải dataset ViSEC từ Hugging Face...")
visec_raw = load_dataset("hustep-lab/ViSEC")
print("Các splits có sẵn:", list(visec_raw.keys()))
print("Số mẫu tổng (train):", len(visec_raw["train"]))

# 2. Map 4 labels cảm xúc gốc → 3 labels (positive, negative, neutral)
print("\n[2/5] Đang chuẩn hóa nhãn cảm xúc từ 4 lớp về 3 lớp...")

def map_emotion_to_3_labels(emotion: str) -> str:
    """Map 4 nhãn ViSEC (happy, neutral, sad, angry) → 3 nhãn (positive, negative, neutral)."""
    emotion = str(emotion).lower().strip()
    if emotion == "happy":
        return "positive"
    if emotion == "neutral":
        return "neutral"
    if emotion in ["sad", "angry"]:
        return "negative"
    return "neutral"

visec_data = []
for ex in visec_raw["train"]:
    mapped_emotion = map_emotion_to_3_labels(ex["emotion"])
    visec_data.append(
        {
            "audio": ex["audio"],
            "emotion": mapped_emotion,
            "emotion_original": ex["emotion"],
            "duration": float(ex["duration"]),
            "accent": ex["accent"],
            "gender": ex["gender"],
        }
    )

visec_df = pd.DataFrame(visec_data)

print("Phân bố nhãn cảm xúc sau khi map:")
print(visec_df["emotion"].value_counts())

print("\nPhân bố accent:")
print(visec_df["accent"].value_counts())

print("\nPhân bố gender:")
print(visec_df["gender"].value_counts())

# 3. Chia train / validation / test với stratify theo nhãn cảm xúc
print("\n[3/5] Đang chia train / validation / test (70% / 15% / 15%) với stratify theo emotion...")

train_df, temp_df = train_test_split(
    visec_df,
    test_size=0.3,
    stratify=visec_df["emotion"],
    random_state=42,
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["emotion"],
    random_state=42,
)

print("Số mẫu:")
print("  Train:", len(train_df))
print("  Val:  ", len(val_df))
print("  Test: ", len(test_df))

print("\nPhân bố emotion trong từng split:")
print("Train:")
print(train_df["emotion"].value_counts())
print("\nVal:")
print(val_df["emotion"].value_counts())
print("\nTest:")
print(test_df["emotion"].value_counts())

# 4. Chuyển về Hugging Face Dataset và cast cột audio về Audio(16000)
print("\n[4/5] Đang chuyển về Hugging Face Dataset và chuẩn hóa cột audio...")

# Hàm helper để giữ lại đúng các cột cần thiết
COLUMNS = ["audio", "emotion", "emotion_original", "duration", "accent", "gender"]

train_ds = Dataset.from_pandas(train_df[COLUMNS]).cast_column("audio", Audio(sampling_rate=16000))
val_ds = Dataset.from_pandas(val_df[COLUMNS]).cast_column("audio", Audio(sampling_rate=16000))
test_ds = Dataset.from_pandas(test_df[COLUMNS]).cast_column("audio", Audio(sampling_rate=16000))

print("Ví dụ 1 mẫu train:")
print({k: train_ds[0][k] for k in ["emotion", "emotion_original", "duration", "accent", "gender"]})

# 5. Lưu các split xuống đĩa và ghi thống kê
print("\n[5/5] Đang lưu các split đã xử lý và ghi thống kê...")

train_path = OUTPUT_DIR / "train"
val_path = OUTPUT_DIR / "val"
test_path = OUTPUT_DIR / "test"

train_ds.save_to_disk(str(train_path))
val_ds.save_to_disk(str(val_path))
test_ds.save_to_disk(str(test_path))

print("Đã lưu:")
print("  - Train:", train_path.resolve())
print("  - Val:  ", val_path.resolve())
print("  - Test: ", test_path.resolve())

train_durations = [ex["duration"] for ex in train_ds]

stats = {
    "train_samples": len(train_ds),
    "val_samples": len(val_ds),
    "test_samples": len(test_ds),
    "total_samples": len(train_ds) + len(val_ds) + len(test_ds),
    "emotion_distribution": visec_df["emotion"].value_counts().to_dict(),
    "avg_duration": float(sum(train_durations) / len(train_durations)),
    "min_duration": float(min(train_durations)),
    "max_duration": float(max(train_durations)),
}

with open(OUTPUT_DIR / "stats.json", "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

print("\nHoàn thành chuẩn bị dữ liệu SER cho bộ ViSEC.")
print("Thống kê chính:")
for k, v in stats.items():
    print(f"  {k}: {v}")

